# Homework 3
CS 510 Large Language Models - Winter 2024
Bradley Thompson

## 1 - Dataset Annotation

To create the dataset, the authors utilized the Twitter API to retrieve 198 million tweets posted between May 2018 and March 2020. The original dataset included tweets in over thirty languages. It was filtered to consider only tweets with at least three tokens and without URLs to exclude bot tweets and spam advertising. Further, to create a balanced dataset for sentiment analysis, the authors made 8 distinct
monolingual datasets. To ensure balance, a maximum number of tweets were established based on the size of the smallest dataset (3,033 tweets for Hindi) -- all other languages' datasets were pruned. The resulting dataset consists of 1,839 training tweets and 870 testing tweets, with a total size of 24,262 tweets. In terms of label distribution, the dataset was created with an equal distribution across the three labels, and the distribution is maintained throughout the train, test and validation sets.

A few potential weaknesses of the annotation process are:
- Intermixing languages with diverse scripts (only Arabic from a separate language family) could make it tough for the model to perform well on sentiment analysis tasks.
- The labeling process itself is subjective, because an emotional interpretation of a given tweet's sentiment comes down to the annotator, and may not accurately track the tweet's actual sentiment. This is exacerbated by cultural differences between language speaking populations.
- Aside from that, there isn't much information in the paper about the actual annotation process, specifically about the annotators themselves, so we can't know how qualified they were to analyze tweet sentiment.



## 2 - Language Diversity

| Language | Family | Resource Level (High / Low)|
|----------|--------|----------------------------|
|Arabic|Afro-Asiatic|Low|
|English|Indo-European|High|
|French|Indo-European|High|
|German|Indo-European|High|
|Hindi|Indo-European|Low|
|Italian|Indo-European|Low|
|Portoguese|Indo-European|Low|
|Spanish|Indo-European|High

Given this information, I believe that the high resource level languages will perform best, and I believe Arabic will perform significantly
worse. This is a result of the overwhelming representation of data in the "Indo-European" family: Because these languages all will share common traits, it's possible that multi-language models will benefit from an understanding built on shared language patterns. Conversely, the only language (again, Arabic) representing an entirely different language family will garner no such benefit.

Aside from these highlights, below I include the languages ranked from highest to lowest, as well as my thoughts on why I ranked each language as such inline:

| Rank | Language | Notes |
|------|----------|-------|
|1|English|Obvious choice; Highest representation in data online|
|2|Spanish|Close call with French/German, but wins because more Spanish speaking countries, so probably more representation|
|3|French|Similar to German, but subjectively, seems like it could be more emotive, which will help with sentiment analysis|
|4|German|Last slot of the high resource languages|
|5|Hindi|Subjectively, I think this would be the highest representation low resource language|
|6|Portoguese|Close behind Hindi, also because of assumed representation online|
|7|Italian|I imagine that this is the lowest representation Indo-European language|
|8|Arabic|Reasons referenced in preceding paragraph|

On second read-through of the assignment, I realized that I only need to use languages from the Indo-European family. I'm leaving in the Arabic analysis because it
still seems relevant as the language is included as a subset of the available data, regardless.

## 3 - Multilingual Sentiment Analysis

To establish a base case before investigating advanced prompting strategies, I started out with simple / regular prompting for the sentiment analysis task. The models we are studying are OpenAI's ChatGPT 3.5 and Google's Gemini.

The prompt that I used for the base case is provided below:

```
Given this json array of strings containing tweets, can you analyze the tweet's sentiment and label it as POSITIVE, or NEGATIVE?
[Array of texts here]
Please organize the texts alongside the result Sentiment label in a python dict so that I can easily turn the results into a pandas dataframe.
```

And I used the following code cells to import the dataset and organize it by label. I then pulled out data for use in subsequent prompting on the web UI for both models.

In [1]:
from datasets import load_dataset

POSITIVE="positive"
NEUTRAL="neutral"
NEGATIVE="negative"

ID_TO_LABEL = {
    0: NEGATIVE,
    1: NEUTRAL,
    2: POSITIVE,
}

dataset = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "english")
target_dataset = dataset["train"]

# Check out what the data looks like:

positive = [ sample["text"] for sample in target_dataset if ID_TO_LABEL[sample["label"]] == POSITIVE ]
neutral = [ sample["text"] for sample in target_dataset if ID_TO_LABEL[sample["label"]] == NEUTRAL ]
negative = [ sample["text"] for sample in target_dataset if ID_TO_LABEL[sample["label"]] == NEGATIVE ]

In [2]:
print("Positives:", positive[:25])
print("Negatives:", negative[:25])
positive[100]

Positives: ['"Frank Gaffrey\\u002c Cliff May\\u002c Steve Emerson: Brilliant. \\""""Looming Threats: Iran\\u002c Hezbollah Hamas\\"""" is the best #cufidc session I\\u2019ve had thus far." ', '"People always forget the fact that Shawn achieved so much in the age of 16 like his 1st single, EP and FIRST album ALL went #1 on charts" ', '"Winnipeg Sun: ""But make no mistake: Janet Jackson played to win. And did."" #UnbreakableWinnipeg #UnbreakableWorldTour ', 'Better be with Kendrick Lamar ', '#EDsummit15 is an opportunity for candidates like Carly Fiorina to flesh out education platforms for the 1st time: ', "Great article in Rolling Stone on Rod Picott. Check it out!  I'll be interviewing Rod on the August 9th episode... ", 'Fianlly gaming review of Moto G3 and it is a solid performer! 7 Graphics heavy games with Moto G 3rd generation! ', 'Ooshma Garg started her 2nd successful company\\u002c Gobble. Watch how she got to one of her biggest investors--Reid Hoffman ', "I'm getting strong m

'The Apple Watch may just change the way we travel. How do you think smart watches will aid our trips? '

In [3]:
# Base case result aggregation (not bothering with storing in an external file)
import pandas as pd

# CHATGPT

# Used the models themselves to arrange the results in this form for easy use in a dataframe.
pos_results = [
    {"Text": "Frank Gaffrey, Cliff May, Steve Emerson: Brilliant. \"Looming Threats: Iran, Hezbollah Hamas\" is the best #cufidc session I’ve had thus far.", "Sentiment": "POSITIVE"},
    {"Text": "People always forget the fact that Shawn achieved so much in the age of 16 like his 1st single, EP and FIRST album ALL went #1 on charts", "Sentiment": "POSITIVE"},
    {"Text": "Winnipeg Sun: \"But make no mistake: Janet Jackson played to win. And did.\" #UnbreakableWinnipeg #UnbreakableWorldTour", "Sentiment": "POSITIVE"},
    {"Text": "Better be with Kendrick Lamar", "Sentiment": "POSITIVE"},
    {"Text": "#EDsummit15 is an opportunity for candidates like Carly Fiorina to flesh out education platforms for the 1st time", "Sentiment": "POSITIVE"},
    {"Text": "Great article in Rolling Stone on Rod Picott. Check it out! I'll be interviewing Rod on the August 9th episode...", "Sentiment": "POSITIVE"},
    {"Text": "Fianlly gaming review of Moto G3 and it is a solid performer! 7 Graphics heavy games with Moto G 3rd generation!", "Sentiment": "POSITIVE"},
    {"Text": "Ooshma Garg started her 2nd successful company, Gobble. Watch how she got to one of her biggest investors--Reid Hoffman", "Sentiment": "POSITIVE"},
    {"Text": "I'm getting strong mail that Eden Hazard will become a Roo tomorrow. Pick 39 & Garlett to wherever the f*ck Hazard plays soccer. Good deal", "Sentiment": "NEGATIVE"},
    {"Text": "@user @user David Cameron is like god & guide to Syrian refugees. God may bless the people like David Cameron", "Sentiment": "POSITIVE"},
    {"Text": "Last tweet of the night, reading will not lose to spurs tomorrow", "Sentiment": "POSITIVE"},
    {"Text": "Not even 20 pages into Paper Towns and the book centers around the night of May 5th. I know its gonna be a good one now! #cincodemayo", "Sentiment": "POSITIVE"},
    {"Text": "Red Sox off to another great start. They lead the Phillies after two innings at Fenway 6-0, scoring 2 in the 1st and 4 more in the second", "Sentiment": "POSITIVE"},
    {"Text": "Watched a Pride and Prejudice play and then the season finale of the 2nd season of Downton Abbey. Tonight is so British.", "Sentiment": "POSITIVE"},
    {"Text": "Just made a plot for a fan fic about How To Rock. It's gonna be called before gravity 5. I'll post it by Friday can't wait for u to read it.", "Sentiment": "POSITIVE"},
    {"Text": "@user @user ha! Probably google. Get some pro ones done for the next time! Off to @user tomorrow! Can't wait", "Sentiment": "POSITIVE"},
    {"Text": "I went to the Polish Festival on Roncesvalles on Saturday awesome time!", "Sentiment": "POSITIVE"},
    {"Text": "@user I had Arian Foster so I'm on 26 already, but having Luck and Hilton puts you in a good place going into NFL Sunday.", "Sentiment": "POSITIVE"},
    {"Text": "Woot! So excited that I get to watch tonight's game. Go Colts!!", "Sentiment": "POSITIVE"},
    {"Text": "Very excited for @user #SummerSlam paperview this Sunday. Man I really hope the Undertaker tombstones Brock Lesnar back to the UFC", "Sentiment": "POSITIVE"},
    {"Text": "A BIG day at Cardiff Airport tomorrow for Iron Maiden fans! Watch this space to find out what's going...", "Sentiment": "POSITIVE"},
    {"Text": "Did you know that 'Janet Jackson' was Trending Topic on Thursday 3 for 8 hours in Calgary? #trndnl", "Sentiment": "POSITIVE"},
    {"Text": "@user Hi Martina - am at St. Patrick's tomorrow for 10:30 Mass. Hope to see you then.", "Sentiment": "POSITIVE"},
    {"Text": "My Sunday nights haven’t been the same since @user has been gone from Breakout Kings. I wonder what he’s doing next? Can’t wait 2 c!", "Sentiment": "POSITIVE"},
    {"Text": ".@LenKasper: \"Bryant has hit some big home runs...\" [Kris Bryant hits a game-tying two-run HR in the 8th]", "Sentiment": "POSITIVE"}
]
neg_results = [
    {"Text": "okay i\\u2019m sorry but TAYLOR SWIFT LOOKS NOTHING LIKE JACKIE O SO STOP COMPARING THE TWO. c\\u2019mon America aren\\u2019t you sick of her yet? (sorry)", "Sentiment": "NEGATIVE"},
    {"Text": "The tragedy of only thinking up hilarious tweets for the Summer Olympics now is that in four years there may be no place for them.", "Sentiment": "NEGATIVE"},
    {"Text": "it looks like a beautiful night to throw myself off the Brooklyn Bridge ---@Tim_Hecht", "Sentiment": "NEGATIVE"},
    {"Text": "I wanna go to the studio with Ulysses n them tomorrow\\u002cbut i cant. #BARS", "Sentiment": "NEGATIVE"},
    {"Text": "@user a bit frustrating. I don\\u2019t think I\\u2019ve added you on my new PSN account. I\\u2019ll do it tomorrow.", "Sentiment": "NEGATIVE"},
    {"Text": "\"I just sat through Kanye West's MTV speech, what the fuck was that...\"", "Sentiment": "NEGATIVE"},
    {"Text": "Hillary's campaign now reset for the 4th time. Adding humor and heart to a person that has #neither #sadtrombone", "Sentiment": "NEGATIVE"},
    {"Text": "Blow to the Lions...Joel Patfull's out for the season after breaking his hand in Sunday's loss to Adelaide. Surgery Wed morn.  #afl", "Sentiment": "NEGATIVE"},
    {"Text": "\"#BritishBuddhu if Rahul Gandhi is really a British Citizen, I'm very concerned about the comedians out there. They may be without jobs\"", "Sentiment": "NEGATIVE"},
    {"Text": "Hulk Hogan picked the wrong time to be an ass the 1st class of WWE wrestlers are dying off like flies around a zapper #RIPRoddyPiper", "Sentiment": "NEGATIVE"},
    {"Text": "\"This weekend on the Fair &amp; Balanced network, Fox News Sunday's guests are crazy conservative Rick Perry and nutty conservative John Kasich.\"", "Sentiment": "NEGATIVE"},
    {"Text": "@user I installed Madden 16 Deluxe last Monday night for PS4 and still haven't received my packs today nor the reward for opening 50", "Sentiment": "NEGATIVE"},
    {"Text": "\"\\""""men tomorrow you will have one of your hardest patrols...CIF turn in\\"""" lets hope i have everything\"", "Sentiment": "NEGATIVE"},
    {"Text": "\"investigative video reveals Planned Parenthood may be committing infanticide, babies born alive, murdered, and sold.\" ACLJ \"with child\" KJV", "Sentiment": "NEGATIVE"},
    {"Text": "\"Yup, guess what? Citizen weren't the happiest supporter last night, Liverpudlian were. The Fact is: finis in 8th, below Everton Asses #LOL\"", "Sentiment": "NEGATIVE"},
    {"Text": "@user you might not wanna come to anatomy tomorrow\\u002c we have a test lol", "Sentiment": "NEGATIVE"},
    {"Text": "Donald Trump: I will be in D.C. on Wednesday,1 PM, in front of the Capitol, to protest the horrible &amp; incompetent deal being made with Iran.", "Sentiment": "NEGATIVE"},
    {"Text": "@user all I can say is that it was very unrealistic. The 1st movie was better-storyline\\u002c dialogue! And of course \\""""The Grey\\"""" thumbs up!", "Sentiment": "NEGATIVE"},
    {"Text": "@user BY HAVING Seth Rollins as number 1? All credibility is lost. May be the worst WWE champion in history! WWE owns yall?", "Sentiment": "NEGATIVE"},
    {"Text": "Satan worshipers align with Planned Parenthood to defend the practice of chopping up babies for profit", "Sentiment": "NEGATIVE"},
    {"Text": "@user may i also remind you Milan  was one the original clubs punished in the scandle", "Sentiment": "NEGATIVE"},
    {"Text": "@user not one word deploring attacks on Charlie Hebdo nor barbaric nature of islam in the 21st C as long as it's sharia compliant", "Sentiment": "NEGATIVE"},
    {"Text": "Saw it late but Carlos Gomez may have passed Ryan Braun in most hated baseball players", "Sentiment": "NEGATIVE"},
    {"Text": "Christians snapchat story makes me want to kill myself..like I feel like a depressed 8th grader going through that emo phase", "Sentiment": "NEGATIVE"},
    {"Text": "Was just talking about Frank Gifford Sat &amp; sadly he dies on Sun. Maybe I'll be talking about @user today. #gopclowncar", "Sentiment": "NEGATIVE"}
]

pos_df = pd.DataFrame(pos_results)
neg_df = pd.DataFrame(neg_results)

In [4]:
# Base case result aggregation (not bothering with storing in an external file)
import pandas as pd

# Gemini

# Couldn't complete.

In [5]:
pos_df

,Text,Sentiment
0,"Frank Gaffrey, Cliff May, Steve Emerson: Brill...",POSITIVE
1,People always forget the fact that Shawn achie...,POSITIVE
2,"Winnipeg Sun: ""But make no mistake: Janet Jack...",POSITIVE
3,Better be with Kendrick Lamar,POSITIVE
4,#EDsummit15 is an opportunity for candidates l...,POSITIVE
5,Great article in Rolling Stone on Rod Picott. ...,POSITIVE
6,Fianlly gaming review of Moto G3 and it is a s...,POSITIVE
7,Ooshma Garg started her 2nd successful company...,POSITIVE
8,I'm getting strong mail that Eden Hazard will ...,NEGATIVE
9,@user @user David Cameron is like god & guide ...,POSITIVE


Right off the bat, ChatGPT performed extremely well without any advanced prompting techniques. It only mislabeled a single sample, and it was not difficult to get it to understand the problem and, further, format the data so that it was easy for me to analyze.

Gemini on the other hand was not able to work with the same base prompt, when I tried the result was "I couldn't complete your request" after a long pause. Further attempts to get Gemini to analyze the sentiment of the texts, without stemming into some of the advanced techniques I intend to evaluate in the next section, all failed. Gemini tried to give me some crappy code to evaluate the sentiment of the texts based on the presence of a few words that it associated into categories (e.g. if text contains "bad" or "hate" then it is negative, if it contains "great" or "love" then it is positive").

So, the base results to compare against are:
- ChatGPT: almost no room for improvement, already at 98% accuracy, 100% precision and 96% recall.
- Gemini: couldn't achieve the task in the base case.

### A - Advanced Prompting Strategies

I decided to focus on two advanced prompting strategies: CoT (kinda auto-CoT) and emotion prompting. I decided to stick to two because there are a decent number of test cases for the assignment given the manny different parts of problem number 3, and because chatGPT already doesn't have much room for improvement.

#### Chain of Thought

My chain of thought prompt was:

```
You're a Twitter employee in charge of analyzing the sentiment of some tweets to determine if they are POSITIVE, or NEGATIVE.
In the following text, break down some key points that point to the tweet being either POSITIVE, or NEGATIVE, and then use those findings
to report the category label (POSITIVE, or NEGATIVE) for the tweet. After providing this information, could you please include a python code
snippet creating a dict containing the input "Text" as well as category label "Sentiment" for each tweet?

Here's the text for you to analyze, in a json array:
[JSON ARRAY HERE]
```

In [6]:
# Chain-of-Thought Result Aggregation
import pandas as pd

# ChatGPT

pos_results = [
    {"Text": "Frank Gaffrey, Cliff May, Steve Emerson: Brilliant. \"Looming Threats: Iran, Hezbollah Hamas\" is the best #cufidc session I’ve had thus far.", "Sentiment": "POSITIVE"},
    {"Text": "People always forget the fact that Shawn achieved so much in the age of 16 like his 1st single, EP and FIRST album ALL went #1 on charts", "Sentiment": "POSITIVE"},
    {"Text": "Winnipeg Sun: \"But make no mistake: Janet Jackson played to win. And did.\" #UnbreakableWinnipeg #UnbreakableWorldTour", "Sentiment": "POSITIVE"},
    {"Text": "Better be with Kendrick Lamar", "Sentiment": "NEUTRAL"},
    {"Text": "#EDsummit15 is an opportunity for candidates like Carly Fiorina to flesh out education platforms for the 1st time:", "Sentiment": "POSITIVE"},
    {"Text": "Great article in Rolling Stone on Rod Picott. Check it out!  I'll be interviewing Rod on the August 9th episode...", "Sentiment": "POSITIVE"},
    {"Text": "Fianlly gaming review of Moto G3 and it is a solid performer! 7 Graphics heavy games with Moto G 3rd generation!", "Sentiment": "POSITIVE"},
    {"Text": "Ooshma Garg started her 2nd successful company, Gobble. Watch how she got to one of her biggest investors--Reid Hoffman", "Sentiment": "POSITIVE"},
    {"Text": "I'm getting strong mail that Eden Hazard will become a Roo tomorrow. Pick 39 & Garlett to wherever the f*ck Hazard plays soccer. Good deal", "Sentiment": "NEGATIVE"},
    {"Text": '@user @user David Cameron is like god & guide to Syrian refugees.God may blees the people like David Cameron', "Sentiment": "POSITIVE"},
    {"Text": 'Last tweet of the night\\u002c reading will not lose to spurs tomorrow', "Sentiment": "NEUTRAL"},
    {"Text": 'Not even 20 pages into Paper Towns and the book centers around the night of May 5th. I know its gonna be a good one now! #cincodemayo', "Sentiment": "POSITIVE"},
    {"Text": '"Red Sox off to another great start. They lead the Phillies after two innings at Fenway 6-0, scoring 2 in the 1st and 4 more in the second"', "Sentiment": "POSITIVE"},
    {"Text": 'Watched a Pride and Prejudice play and then the season finale of the 2nd season of Downton Abbey. Tonight is so British.', "Sentiment": "POSITIVE"},
    {"Text": "Just made a plot for a fan fic about How To Rock. It's gonna be called before gravity 5. I'll post it by Friday can't wait for u to read it.", "Sentiment": "POSITIVE"},
    {"Text": "@user @user ha! Probably google. Get some pro ones done for the next time! Off to @user tomorrow! Can't wait", "Sentiment": "POSITIVE"},
    {"Text": 'I went to the Polish Festival on Roncesvalles on Saturday awesome time!', "Sentiment": "POSITIVE"},
    {"Text": '@user I had Arian Foster so I\'m on 26 already, but having Luck and Hilton puts you in a good place going into NFL Sunday.', "Sentiment": "POSITIVE"},
    {"Text": "Woot! So excited that I get to watch tonight's game. Go Colts!!", "Sentiment": "POSITIVE"},
    {"Text": 'Very excited for @user #SummerSlam paperview this Sunday. Man I really hope the Undertaker tombstones Brock Lesnar back to the UFC', "Sentiment": "POSITIVE"},
    {"Text": "A BIG day at Cardiff Airport tomorrow for Iron Maiden fans! Watch this space to find out what's going...", "Sentiment": "POSITIVE"},
    {"Text": 'Did you know that \'Janet Jackson\' was Trending Topic on Thursday 3 for 8 hours in Calgary? #trndnl', "Sentiment": "POSITIVE"},
    {"Text": "@user Hi Martina - am at St. Patrick's tomorrow for 10:30 Mass. Hope to see you then.", "Sentiment": "POSITIVE"},
    {"Text": 'My Sunday nights haven\\u2019t been the same since @user has been gone from Breakout Kings. I wonder what he\\u2019s doing next? Can\\u2019t wait 2 c!', "Sentiment": "POSITIVE"},
    {"Text": '".@LenKasper: ""Bryant has hit some big home runs..."" [Kris Bryant hits a game-tying two-run HR in the 8th]"', "Sentiment": "POSITIVE"}
]
neg_results = [
    {'Text': 'okay i\\u2019m sorry but TAYLOR SWIFT LOOKS NOTHING LIKE JACKIE O...', 'Sentiment': 'NEGATIVE'},
    {'Text': 'The tragedy of only thinking up hilarious tweets for the Summer Olympics now...', 'Sentiment': 'POSITIVE'},
    {'Text': 'it looks like a beautiful night to throw myself off the Brooklyn Bridge ---@Tim_Hecht', 'Sentiment': 'NEGATIVE'},
    {'Text': 'I wanna go to the studio with Ulysses n them tomorrow\\u002cbut i cant. #BARS', 'Sentiment': 'NEUTRAL'},
    {'Text': '@user a bit frustrating. I don\\u2019t think I\\u2019ve added you on my new PSN account. I\\u2019ll do it tomorrow.', 'Sentiment': 'NEUTRAL'},
    {'Text': '"I just sat through Kanye West\'s MTV speech, what the fuck was that..."', 'Sentiment': 'NEGATIVE'},
    {'Text': "Hillary's campaign now reset for the 4th time. Adding humor and heart to a person that has #neither #sadtrombone", 'Sentiment': 'NEGATIVE'},
    {'Text': "Blow to the Lions...Joel Patfull's out for the season after breaking his hand in Sunday's loss to Adelaide. Surgery Wed morn.  #afl", 'Sentiment': 'NEGATIVE'},
    {'Text': '"#BritishBuddhu if Rahul Gandhi is really a British Citizen, I\'m very concerned about the comedians out there. They may be without jobs"', 'Sentiment': 'NEGATIVE'},
    {'Text': 'Hulk Hogan picked the wrong time to be an ass the 1st class of WWE wrestlers are dying off like flies around a zapper #RIPRoddyPiper', 'Sentiment': 'NEGATIVE'},
    {'Text': '"This weekend on the Fair &amp; Balanced network, Fox News Sunday\'s guests are crazy conservative Rick Perry and nutty conservative John Kasich."', 'Sentiment': 'NEGATIVE'},
    {'Text': "@user I installed Madden 16 Deluxe last Monday night for PS4 and still haven't received my packs today nor the reward for opening 50", 'Sentiment': 'NEGATIVE'},
    {'Text': '"\\""""men tomorrow you will have one of your hardest patrols...CIF turn in\\"""" lets hope i have everything"', 'Sentiment': 'NEUTRAL'},
    {'Text': '"investigative video reveals Planned Parenthood may be committing infanticide, babies born alive, murdered, and sold." ACLJ "with child" KJV', 'Sentiment': 'NEGATIVE'},
    {'Text': '"Yup, guess what? Citizen weren\'t the happiest supporter last night, Liverpudlian were. The Fact is: finis in 8th, below Everton Asses #LOL"', 'Sentiment': 'NEGATIVE'},
    {'Text': '@user you might not wanna come to anatomy tomorrow\\u002c we have a test lol', 'Sentiment': 'NEUTRAL'},
    {'Text': 'Donald Trump: I will be in D.C. on Wednesday,1 PM, in front of the Capitol, to protest the horrible &amp; incompetent deal being made with Iran.', 'Sentiment': 'NEGATIVE'},
    {'Text': '@user all I can say is that it was very unrealistic. The 1st movie was better-storyline\\u002c dialogue! And of course \\""""The Grey\\"""" thumbs up!', 'Sentiment': 'NEGATIVE'},
    {'Text': '@user BY HAVING Seth Rollins as number 1? All credibility is lost. May be the worst WWE champion in history! WWE owns yall?', 'Sentiment': 'NEGATIVE'},
    {'Text': 'Satan worshipers align with Planned Parenthood to defend the practice of chopping up babies for profit', 'Sentiment': 'NEGATIVE'},
    {'Text': '@user may i also remind you Milan  was one the original clubs punished in the scandle', 'Sentiment': 'NEGATIVE'},
    {'Text': "@user not one word deploring attacks on Charlie Hebdo nor barbaric nature of islam in the 21st C as long as it's sharia compliant", 'Sentiment': 'NEGATIVE'},
    {'Text': 'Saw it late but Carlos Gomez may have passed Ryan Braun in most hated baseball players', 'Sentiment': 'NEGATIVE'},
    {'Text': 'Christians snapchat story makes me want to kill myself..like I feel like a depressed 8th grader going through that emo phase', 'Sentiment': 'NEGATIVE'},
    {"Text": "Was just talking about Frank Gifford Sat &amp; sadly he dies on Sun. Maybe I'll be talking about @user today. #gopclowncar", 'Sentiment': 'NEUTRAL'}
]

pos_df = pd.DataFrame(pos_results)
neg_df = pd.DataFrame(neg_results)

In [7]:
def count_label(df, label):
    return len([ val for val in df["Sentiment"] if val == label ])

print(f"True positive: {count_label(pos_df, 'POSITIVE')} | False positive: {count_label(neg_df, 'POSITIVE')}")
print(f"True negative: {count_label(neg_df, 'NEGATIVE')} | False negative: {count_label(pos_df, 'NEGATIVE')}")
print(f"Neutral: {count_label(pos_df, 'NEUTRAL') + count_label(neg_df, 'NEUTRAL')}")

True positive: 22 | False positive: 1
True negative: 19 | False negative: 1
Neutral: 7


Chain of thought prompting, interestingly, made ChatGPT perform worse. It seems the the additional information confused the model, as the inclusion of the key points it came up with introduced nuance to the sentiment analysis task. Subjectively, it seems that the samples that caused it to introduce a third, unspecified label for (NEUTRAL) were all fairly neutral to be in the corresponding set.

For example, from the positive set, ChatGPT categorized "Better be with Kendrick Lamar" as neutral. The annotators classified it is positive. I think positive is more correct based on assumed cultural context; however, the language used here leaves it open to interpretation and the tone could change wildly based on context. For example, I'd consider this a negative sentiment tweet if it was prefaced with something like "Miss me with this modern trash mumble rap -- Better be with Kendrick Lamar." I think given the wild swing in sentiment possible from this tweet, which honestly seems like a snippet of a larger post anyway, the model was justified in labeling this as NEUTRAL. While the example I provided was not the actual tweet, it still goes to show that the model points out and will potentially be more focused on more nuanced language patterns given the chain-of-thought that it builds.

Another example to further this point, and which also highlights the subjectivity dataset annotation, is actually the direct chain-of-thought provided for the following sample:

"The tragedy of only thinking up hilarious tweets for the Summer Olympics now is that in four years there may be no place for them"

ChatGPT's label: Positive

Reasoning: The tweet is humorous and acknowledges the limitation of tweeting about the Summer Olympics only every four years.

After reading ChatGPT's justification, I honestly agree more with it than the annotator. I think this simply highlights the difficulty of sentiment analysis given the weaknesses discussed at the start of the assignment, and indicates that even though my statistical analysis (Accuracy, precision, recall) might prove worse over time as I try out these advanced prompting techniques -- because of the dataset annotation -- the true performance of the models may actually improve in a more holistic sense (not trusting the annotator's word on a tweet's sentiment, but arriving at a more accurate categorization).


In [8]:
# Chain-of-Thought Result Aggregation
import pandas as pd

# Gemini

pos_results = [
    {"Text": "Frank Gaffrey, Cliff May, Steve Emerson: Brilliant. \"Looming Threats: Iran, Hezbollah Hamas\" is the best #cufidc session I’ve had thus far.", "Sentiment": "POSITIVE"},
    {"Text": "People always forget the fact that Shawn achieved so much in the age of 16 like his 1st single, EP and FIRST album ALL went #1 on charts", "Sentiment": "POSITIVE"},
    {"Text": "Winnipeg Sun: \"But make no mistake: Janet Jackson played to win. And did.\" #UnbreakableWinnipeg #UnbreakableWorldTour", "Sentiment": "POSITIVE"},
    {"Text": "Better be with Kendrick Lamar", "Sentiment": "POSITIVE"},
    {"Text": "#EDsummit15 is an opportunity for candidates like Carly Fiorina to flesh out education platforms for the 1st time", "Sentiment": "NEUTRAL"},
    {"Text": "Great article in Rolling Stone on Rod Picott. Check it out! I'll be interviewing Rod on the August 9th episode...", "Sentiment": "POSITIVE"},
    {"Text": "Fianlly gaming review of Moto G3 and it is a solid performer! 7 Graphics heavy games with Moto G 3rd generation!", "Sentiment": "POSITIVE"},
    {"Text": "Ooshma Garg started her 2nd successful company, Gobble. Watch how she got to one of her biggest investors--Reid Hoffman", "Sentiment": "POSITIVE"},
    {"Text": "I'm getting strong mail that Eden Hazard will become a Roo tomorrow. Pick 39 & Garlett to wherever the f*ck Hazard plays soccer. Good deal", "Sentiment": "MIXED"},
    {"Text": "@user @user David Cameron is like god & guide to Syrian refugees. God may bless the people like David Cameron", "Sentiment": "POSITIVE"},
    {"Text": "Last tweet of the night, reading will not lose to spurs tomorrow", "Sentiment": "POSITIVE"},
    {"Text": "Not even 20 pages into Paper Towns and the book centers around the night of May 5th. I know its gonna be a good one now! #cincodemayo", "Sentiment": "POSITIVE"},
    {"Text": "Red Sox off to another great start. They lead the Phillies after two innings at Fenway 6-0, scoring 2 in the 1st and 4 more in the second", "Sentiment": "POSITIVE"},
    {"Text": "Watched a Pride and Prejudice play and then the season finale of the 2nd season of Downton Abbey. Tonight is so British.", "Sentiment": "POSITIVE"},
    {"Text": "Just made a plot for a fan fic about How To Rock. It's gonna be called before gravity 5. I'll post it by Friday can't wait for u to read it.", "Sentiment": "POSITIVE"},
    {"Text": "@user @user ha! Probably google. Get some pro ones done for the next time! Off to @user tomorrow! Can't wait", "Sentiment": "MIXED"},
    {"Text": "I went to the Polish Festival on Roncesvalles on Saturday awesome time!", "Sentiment": "POSITIVE"},
    {"Text": "@user I had Arian Foster so I'm on 26 already, but having Luck and Hilton puts you in a good place going into NFL Sunday.", "Sentiment": "POSITIVE"},
    {"Text": "Woot! So excited that I get to watch tonight's game. Go Colts!!", "Sentiment": "POSITIVE"},
    {"Text": "Very excited for @user #SummerSlam paperview this Sunday. Man I really hope the Undertaker tombstones Brock Lesnar back to the UFC", "Sentiment": "POSITIVE"},
    {"Text": "A BIG day at Cardiff Airport tomorrow for Iron Maiden fans! Watch this space to find out what's going...", "Sentiment": "POSITIVE"},
    {"Text": "Did you know that 'Janet Jackson' was Trending Topic on Thursday 3 for 8 hours in Calgary? #trndnl", "Sentiment": "POSITIVE"},
    {"Text": "@user Hi Martina - am at St. Patrick's tomorrow for 10:30 Mass. Hope to see you then.", "Sentiment": "POSITIVE"},
    {"Text": "My Sunday nights haven’t been the same since @user has been gone from Breakout Kings. I wonder what he’s doing next? Can’t wait 2 c!", "Sentiment": "NEGATIVE"},
    {"Text": ".@LenKasper: \"Bryant has hit some big home runs...\" [Kris Bryant hits a game-tying two-run HR in the 8th]", "Sentiment": "POSITIVE"}
]
neg_results = [
    {"Text": "okay i\\u2019m sorry but TAYLOR SWIFT LOOKS NOTHING LIKE JACKIE O SO STOP COMPARING THE TWO. c\\u2019mon America aren\\u2019t you sick of her yet? (sorry)", "Sentiment": "NEGATIVE"},
    {"Text": "The tragedy of only thinking up hilarious tweets for the Summer Olympics now is that in four years there may be no place for them.", "Sentiment": "NEGATIVE"},
    {"Text": "it looks like a beautiful night to throw myself off the Brooklyn Bridge ---@Tim_Hecht", "Sentiment": "NEGATIVE"},
    {"Text": "I wanna go to the studio with Ulysses n them tomorrow\\u002cbut i cant. #BARS", "Sentiment": "NEGATIVE"},
    {"Text": "@user a bit frustrating. I don\\u2019t think I\\u2019ve added you on my new PSN account. I\\u2019ll do it tomorrow.", "Sentiment": "NEGATIVE"},
    {"Text": "\"I just sat through Kanye West's MTV speech, what the fuck was that...\"", "Sentiment": "NEGATIVE"},
    {"Text": "Hillary's campaign now reset for the 4th time. Adding humor and heart to a person that has #neither #sadtrombone", "Sentiment": "NEGATIVE"},
    {"Text": "Blow to the Lions...Joel Patfull's out for the season after breaking his hand in Sunday's loss to Adelaide. Surgery Wed morn.  #afl", "Sentiment": "NEGATIVE"},
    {"Text": "\"#BritishBuddhu if Rahul Gandhi is really a British Citizen, I'm very concerned about the comedians out there. They may be without jobs\"", "Sentiment": "NEGATIVE"},
    {"Text": "Hulk Hogan picked the wrong time to be an ass the 1st class of WWE wrestlers are dying off like flies around a zapper #RIPRoddyPiper", "Sentiment": "NEGATIVE"},
    {"Text": "\"This weekend on the Fair &amp; Balanced network, Fox News Sunday's guests are crazy conservative Rick Perry and nutty conservative John Kasich.\"", "Sentiment": "NEGATIVE"},
    {"Text": "@user I installed Madden 16 Deluxe last Monday night for PS4 and still haven't received my packs today nor the reward for opening 50", "Sentiment": "NEGATIVE"},
    {"Text": "\"\\""""men tomorrow you will have one of your hardest patrols...CIF turn in\\"""" lets hope i have everything\"", "Sentiment": "NEGATIVE"},
    {"Text": "\"investigative video reveals Planned Parenthood may be committing infanticide, babies born alive, murdered, and sold.\" ACLJ \"with child\" KJV", "Sentiment": "NEGATIVE"},
    {"Text": "\"Yup, guess what? Citizen weren't the happiest supporter last night, Liverpudlian were. The Fact is: finis in 8th, below Everton Asses #LOL\"", "Sentiment": "NEGATIVE"},
    {"Text": "@user you might not wanna come to anatomy tomorrow\\u002c we have a test lol", "Sentiment": "NEGATIVE"},
    {"Text": "Donald Trump: I will be in D.C. on Wednesday,1 PM, in front of the Capitol, to protest the horrible &amp; incompetent deal being made with Iran.", "Sentiment": "NEGATIVE"},
    {"Text": "@user all I can say is that it was very unrealistic. The 1st movie was better-storyline\\u002c dialogue! And of course \\""""The Grey\\"""" thumbs up!", "Sentiment": "NEGATIVE"},
    {"Text": "@user BY HAVING Seth Rollins as number 1? All credibility is lost. May be the worst WWE champion in history! WWE owns yall?", "Sentiment": "NEGATIVE"},
    {"Text": "Satan worshipers align with Planned Parenthood to defend the practice of chopping up babies for profit", "Sentiment": "NEGATIVE"},
    {"Text": "@user may i also remind you Milan  was one the original clubs punished in the scandle", "Sentiment": "NEGATIVE"},
    {"Text": "@user not one word deploring attacks on Charlie Hebdo nor barbaric nature of islam in the 21st C as long as it's sharia compliant", "Sentiment": "NEGATIVE"},
    {"Text": "Saw it late but Carlos Gomez may have passed Ryan Braun in most hated baseball players", "Sentiment": "NEGATIVE"},
    {"Text": "Christians snapchat story makes me want to kill myself..like I feel like a depressed 8th grader going through that emo phase", "Sentiment": "NEGATIVE"},
    {"Text": "Was just talking about Frank Gifford Sat &amp; sadly he dies on Sun. Maybe I'll be talking about @user today. #gopclowncar", "Sentiment": "NEGATIVE"}
]

pos_df = pd.DataFrame(pos_results)
neg_df = pd.DataFrame(neg_results)

In [9]:
def count_label(df, label):
    return len([ val for val in df["Sentiment"] if val == label ])

print(f"True positive: {count_label(pos_df, 'POSITIVE')} | False positive: {count_label(neg_df, 'POSITIVE')}")
print(f"True negative: {count_label(neg_df, 'NEGATIVE')} | False negative: {count_label(pos_df, 'NEGATIVE')}")
print(f"Neutral/Mixed: {count_label(pos_df, 'NEUTRAL') + count_label(neg_df, 'NEUTRAL') + count_label(pos_df, 'MIXED') + count_label(neg_df, 'MIXED')}")

True positive: 21 | False positive: 0
True negative: 25 | False negative: 1
Neutral/Mixed: 3


Chain of thought prompting resulted in a marked improvement in performance for Gemini, in that it was able to complete the task to some extent. Gemini was not able to understand my instructions to help speed up data collection, choosing to provide some bad sentiment analysis code snippets instead; however, it was able to at least provide categories for each text, which I could then copy down manually. Interestingly, it added a fourth label "Mixed" which also wasn't mentioned in the prompt. I grouped in the mixed samples with neutral.

Gemini was relatively accurate across the samples given this strategy, and again like ChatGPT, I think in the instances that it was wrong, it was potentially more accurate than the annotators in reality. Here's an example:

"@user @user ha! Probably google. Get some pro ones done for the next time! Off to @user tomorrow! Can't wait"

Gemini's label: Mixed

Reasoning: Makes fun of someone's photos but expresses excitement to meet someone else.

I believe that Gemini was very capable at capturing the nuance of this text, and potential divergence in intended tone based on context. The annotator it seems was very rash to consider this a solely positive tweet; Perhaps Gemini is right and this tweet was actually making fun of the quality of someone's photos. We can't really tell, but that further captures the fact that there's nuance to this sample which is not captured by the annotators label. 

#### Emotion Prompting

My prompt for emotion prompting was a slightly altered version of the base-case prompt:

```
I really need to complete this important assignment or else I will FAIL SCHOOL! I have this json array of strings containing tweets, can you analyze each tweet's sentiment and label it as POSITIVE, or NEGATIVE? I'm super tight on time, and I'd really appreciate it if you could provide a python dictionary containing the input "Text" in one field, and your categorized label (POSITIVE or NEGATIVE) in a "Sentiment" field. An example output dict would look like this:

{"Text": "a tweet", "Sentiment": "Positive"}

Please help me and paste the list of python dicts containing your analysis in response to this input json array:
[Array of texts here]
```


In [17]:
# Emotion Prompting Result Aggregation
import pandas as pd

# ChatGPT
pos_results = [
    {"Text": '"Frank Gaffrey\\u002c Cliff May\\u002c Steve Emerson: Brilliant. \\""""Looming Threats: Iran\\u002c Hezbollah Hamas\\"""" is the best #cufidc session I\\u2019ve had thus far."', "Sentiment": "POSITIVE"},
    {"Text": '"People always forget the fact that Shawn achieved so much in the age of 16 like his 1st single, EP and FIRST album ALL went #1 on charts"', "Sentiment": "POSITIVE"},
    {"Text": '"Winnipeg Sun: ""But make no mistake: Janet Jackson played to win. And did."" #UnbreakableWinnipeg #UnbreakableWorldTour', "Sentiment": "POSITIVE"},
    {"Text": 'Better be with Kendrick Lamar', "Sentiment": "POSITIVE"},
    {"Text": '#EDsummit15 is an opportunity for candidates like Carly Fiorina to flesh out education platforms for the 1st time:', "Sentiment": "POSITIVE"},
    {"Text": "Great article in Rolling Stone on Rod Picott. Check it out!  I'll be interviewing Rod on the August 9th episode...", "Sentiment": "POSITIVE"},
    {"Text": 'Fianlly gaming review of Moto G3 and it is a solid performer! 7 Graphics heavy games with Moto G 3rd generation!', "Sentiment": "POSITIVE"},
    {"Text": 'Ooshma Garg started her 2nd successful company\\u002c Gobble. Watch how she got to one of her biggest investors--Reid Hoffman', "Sentiment": "POSITIVE"},
    {"Text": "I'm getting strong mail that Eden Hazard will become a Roo tomorrow. Pick 39 & Garlett to wherever the f*ck Hazard plays soccer. Good deal ", "Sentiment": "POSITIVE"},
    {"Text": '@user @user David Cameron is like god &amp; guide to Syrian refugees.God may blees the people like David Cameron ', "Sentiment": "POSITIVE"},
    {"Text": 'Last tweet of the night\\u002c reading will not lose to spurs tomorrow', "Sentiment": "POSITIVE"},
    {"Text": 'Not even 20 pages into Paper Towns and the book centers around the night of May 5th. I know its gonna be a good one now! #cincodemayo ', "Sentiment": "POSITIVE"},
    {"Text": '"Red Sox off to another great start. They lead the Phillies after two innings at Fenway 6-0, scoring 2 in the 1st and 4 more in the second"', "Sentiment": "POSITIVE"},
    {"Text": 'Watched a Pride and Prejudice play and then the season finale of the 2nd season of Downton Abbey. Tonight is so British. ', "Sentiment": "POSITIVE"},
    {"Text": "Just made a plot for a fan fic about How To Rock. It's gonna be called before gravity 5. I'll post it by Friday can't wait for u to read it. ", "Sentiment": "POSITIVE"},
    {"Text": "@user @user ha! Probably google. Get some pro ones done for the next time! Off to @user tomorrow! Can't wait ", "Sentiment": "POSITIVE"},
    {"Text": 'I went to the Polish Festival on Roncesvalles on Saturday awesome time! ', "Sentiment": "POSITIVE"},
    {"Text": '@user I had Arian Foster so I\'m on 26 already, but having Luck and Hilton puts you in a good place going into NFL Sunday." ', "Sentiment": "POSITIVE"},
    {"Text": "Woot!  So excited that I get to watch tonight's game. Go Colts!! ", "Sentiment": "POSITIVE"},
    {"Text": 'Very excited for @user #SummerSlam paperview this Sunday. Man I really hope the Undertaker tombstones Brock Lesnar back to the UFC ', "Sentiment": "POSITIVE"},
    {"Text": "A BIG day at Cardiff Airport tomorrow for Iron Maiden fans! Watch this space to find out what's going... ", "Sentiment": "POSITIVE"},
    {"Text": "Did you know that 'Janet Jackson' was Trending Topic on Thursday 3 for 8 hours in Calgary? #trndnl ", "Sentiment": "POSITIVE"},
    {"Text": "@user Hi Martina - am at St. Patrick's tomorrow for 10:30 Mass.  Hope to see you then. ", "Sentiment": "POSITIVE"},
    {"Text": 'My Sunday nights haven\\u2019t been the same since @user has been gone from Breakout Kings. I wonder what he\\u2019s doing next? Can\\u2019t wait 2 c! ', "Sentiment": "POSITIVE"},
    {"Text": '".@LenKasper: ""Bryant has hit some big home runs...""  [Kris Bryant hits a game-tying two-run HR in the 8th]" ', "Sentiment": "POSITIVE"}
]
neg_results = [
    {"Text": 'okay i\\u2019m sorry but TAYLOR SWIFT LOOKS NOTHING LIKE JACKIE O SO STOP COMPARING THE TWO. c\\u2019mon America aren\\u2019t you sick of her yet? (sorry)', "Sentiment": "NEGATIVE"},
    {"Text": 'The tragedy of only thinking up hilarious tweets for the Summer Olympics now is that in four years there may be no place for them.', "Sentiment": "POSITIVE"},
    {"Text": 'it looks like a beautiful night to throw myself off the Brooklyn Bridge ---@Tim_Hecht', "Sentiment": "NEGATIVE"},
    {"Text": 'I wanna go to the studio with Ulysses n them tomorrow\\u002cbut i cant. #BARS', "Sentiment": "NEUTRAL"},
    {"Text": '@user a bit frustrating. I don\\u2019t think I\\u2019ve added you on my new PSN account. I\\u2019ll do it tomorrow.', "Sentiment": "NEUTRAL"},
    {"Text": '"I just sat through Kanye West\'s MTV speech, what the fuck was that..."', "Sentiment": "NEGATIVE"},
    {"Text": "Hillary's campaign now reset for the 4th time. Adding humor and heart to a person that has #neither #sadtrombone", "Sentiment": "NEGATIVE"},
    {"Text": "Blow to the Lions...Joel Patfull's out for the season after breaking his hand in Sunday's loss to Adelaide. Surgery Wed morn.  #afl", "Sentiment": "NEGATIVE"},
    {"Text": '"#BritishBuddhu if Rahul Gandhi is really a British Citizen, I\'m very concerned about the comedians out there. They may be without jobs"', "Sentiment": "NEGATIVE"},
    {"Text": 'Hulk Hogan picked the wrong time to be an ass the 1st class of WWE wrestlers are dying off like flies around a zapper #RIPRoddyPiper', "Sentiment": "NEGATIVE"},
    {"Text": '"This weekend on the Fair &amp; Balanced network, Fox News Sunday\'s guests are crazy conservative Rick Perry and nutty conservative John Kasich."', "Sentiment": "NEGATIVE"},
    {"Text": "@user I installed Madden 16 Deluxe last Monday night for PS4 and still haven't received my packs today nor the reward for opening 50 ", "Sentiment": "NEGATIVE"},
    {"Text": '"\\""""men tomorrow you will have one of your hardest patrols...CIF turn in\\"""" lets hope i have everything"', "Sentiment": "NEUTRAL"},
    {"Text": '"investigative video reveals Planned Parenthood may be committing infanticide, babies born alive, murdered, and sold." ACLJ "with child" KJV', "Sentiment": "NEGATIVE"},
    {"Text": '"Yup, guess what? Citizen weren\'t the happiest supporter last night, Liverpudlian were. The Fact is: finis in 8th, below Everton Asses #LOL"', "Sentiment": "NEGATIVE"},
    {"Text": '@user you might not wanna come to anatomy tomorrow\\u002c we have a test lol ', "Sentiment": "NEUTRAL"},
    {"Text": 'Donald Trump: I will be in D.C. on Wednesday,1 PM, in front of the Capitol, to protest the horrible &amp; incompetent deal being made with Iran.', "Sentiment": "NEGATIVE"},
    {"Text": '@user all I can say is that it was very unrealistic. The 1st movie was better-storyline\\u002c dialogue! And of course \\""""The Grey\\"""" thumbs up!" ', "Sentiment": "NEGATIVE"},
    {"Text": '@user BY HAVING Seth Rollins as number 1? All credibility is lost. May be the worst WWE champion in history! WWE owns yall? ', "Sentiment": "NEGATIVE"},
    {"Text": 'Satan worshipers align with Planned Parenthood to defend the practice of chopping up babies for profit ', "Sentiment": "NEGATIVE"},
    {"Text": '@user may i also remind you Milan  was one the original clubs punished in the scandle ', "Sentiment": "NEGATIVE"},
    {"Text": "@user not one word deploring attacks on Charlie Hebdo nor barbaric nature of islam in the 21st C as long as it's sharia compliant ", "Sentiment": "NEGATIVE"},
    {"Text": 'Saw it late but Carlos Gomez may have passed Ryan Braun in most hated baseball players ', "Sentiment": "NEGATIVE"},
    {"Text": 'Christians snapchat story makes me want to kill myself..like I feel like a depressed 8th grader going through that emo phase ', "Sentiment": "NEGATIVE"},
    {"Text": "Was just talking about Frank Gifford Sat &amp; sadly he dies on Sun. Maybe I'll be talking about @user today. #gopclowncar ", "Sentiment": "NEUTRAL"}
]

pos_df = pd.DataFrame(pos_results)
neg_df = pd.DataFrame(neg_results)
neg_df

,Text,Sentiment
0,okay i\u2019m sorry but TAYLOR SWIFT LOOKS NOT...,NEGATIVE
1,The tragedy of only thinking up hilarious twee...,POSITIVE
2,it looks like a beautiful night to throw mysel...,NEGATIVE
3,I wanna go to the studio with Ulysses n them t...,NEUTRAL
4,@user a bit frustrating. I don\u2019t think I\...,NEUTRAL
5,"""I just sat through Kanye West's MTV speech, w...",NEGATIVE
6,Hillary's campaign now reset for the 4th time....,NEGATIVE
7,Blow to the Lions...Joel Patfull's out for the...,NEGATIVE
8,"""#BritishBuddhu if Rahul Gandhi is really a Br...",NEGATIVE
9,Hulk Hogan picked the wrong time to be an ass ...,NEGATIVE


In [15]:
def count_label(df, label):
    return len([ val for val in df["Sentiment"] if val == label ])

print(f"True positive: {count_label(pos_df, 'POSITIVE')} | False positive: {count_label(neg_df, 'POSITIVE')}")
print(f"True negative: {count_label(neg_df, 'NEGATIVE')} | False negative: {count_label(pos_df, 'NEGATIVE')}")
print(f"Neutral/Mixed: {count_label(pos_df, 'NEUTRAL') + count_label(neg_df, 'NEUTRAL') + count_label(pos_df, 'MIXED') + count_label(neg_df, 'MIXED')}")

True positive: 25 | False positive: 1
True negative: 19 | False negative: 0
Neutral/Mixed: 5


While ChatGPT's performance was statistically hindered, it does still seem to have become more capable in understanding language sentiment nuance in it's findings given the emotional prompt. It flagged many of the same samples in the negative subset that both models pointed out as naunced (neutral or mixed) in chain-of-thought prompting, when the models expressed reasoning as to why the models may not be entirely correct in a category of POSITIVE or NEGATIVE. This exemplifies that the models do have some implicit understanding of the semantics behind the language they analyze, even when not asked to explicitly generate an explanation of those semantics.

Interestingly, the added subjective importance of the emotional prompt has maybe caused the model to spend more computation on evaluating the sentiment of the statements. It would be interesting to spend more dedicated time researching some way to identify the difference in characteristics the model cares about under the different prompt paradigms. For example, when there's no emotional importance expressed, does the model only analyze sentiment based on simple heuristics (contains x, y, z words), and then with the importance of the result stressed, does the model analyze more complex concepts like tone, speech target, context?

In [12]:
# Emotion Prompting Result Aggregation
import pandas as pd

# Gemini

# Unable to complete

Again, Gemini was not able to perform the sentiment analysis task at all. It instead tried to generate a dumb sentiment analysis script in Python that used word presence checks to determine sentiment. The model itself did no sentiment analysis. Gemini seems to need a more thorough prompting strategy to understand the intent of the task.

### B - Multilingual Prompting

For this task, I decided to investigate english prompts compared to spanish prompts on the spanish dataset. I chose to only compare the past results to Spanish because it's the second language that I have the best understanding of, so I don't have to entirely trust Google Translate to determine the language used in the prompt in the latter half of this task.

#### English prompt on Spanish data

In an attempt to have something to compare between the two models, I decided to extend my findings from part A to use a combination of chain-of-thought prompting -- to a certain extent -- as well as augment the english prompt with clear parameters explaining that the input data would be in Spanish, to try and get the models to analyze the sentiment in the Spanish data.

The prompt that I used is provided below:

```
You're a Twitter employee in charge of analyzing the sentiment of some tweets to determine if they are POSITIVE, or NEGATIVE.
In the following text, break down some key points that point to the tweet being either POSITIVE, or NEGATIVE, and then use those findings
to report the category label (POSITIVE, or NEGATIVE) for the tweet. Using your findings, could you please include a python code
snippet creating a dict associating the input text with your "Sentiment" label. Each dict should look like this:

{"Text": "The input tweet here", "Sentiment": "Your label here"}

And again, the labels that you can choose are: [POSITIVE, NEGATIVE]

Note that all input tweets are in Spanish, so you need to understand the meaning of these spanish texts and then provide english labels and explain your reasoning in english.

Here's the text for you to analyze, in a json array:
[JSON ARRAY HERE]
```

In [18]:
from datasets import load_dataset

POSITIVE="positive"
NEUTRAL="neutral"
NEGATIVE="negative"

ID_TO_LABEL = {
    0: NEGATIVE,
    1: NEUTRAL,
    2: POSITIVE,
}

dataset = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "spanish")
target_dataset = dataset["train"]

# Check out what the data looks like:

positive = [ sample["text"] for sample in target_dataset if ID_TO_LABEL[sample["label"]] == POSITIVE ]
neutral = [ sample["text"] for sample in target_dataset if ID_TO_LABEL[sample["label"]] == NEUTRAL ]
negative = [ sample["text"] for sample in target_dataset if ID_TO_LABEL[sample["label"]] == NEGATIVE ]

Generating train split:   0%|          | 0/1839 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/324 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/870 [00:00<?, ? examples/s]

In [19]:
print("Positives:", positive[:25])
print("Negatives:", negative[:25])

Positives: ['Esto de estar feliz mola', '@user @user @user con una pequeña donación hará felices a miles de chicas que no tienen  #asociacionmariloli', '@user aquí tienes a mi bae aka una egipcia preciosa que esta aprendiendo español. Os llevareis bien  @user', '#2AñosDeLegiónHolk Llevo como unos 8 meses Talvez menos o mas no sé Me enseñaron y demostraron que LH es algo magico', 'Se echa de menos  #digital #art #life @ Escuela Superior de Arte y Tecnología (ESAT) http', '@user No. tampoco soy cursi', '@user     Ya está preparando cosillas nuevas jeje', 'Hoy toca escuchar a #MiguelRios con otros grandes artistas...  #Insurrección...', 'Voy a jugar mi primera ranked de 10. Hmm no se si la liaré o no, pero bronce V me espera y mañana jugaré poco o nada, so a jugársela', 'Sin duda @user fue una experiencia preciosa  @user @user Y habrá más!', 'una Go-Pro de calidad! proteccion bajo el agua y soporte de cabeza    porq ago tantas caras...OK EN FIN!', 'Que dicha que hay trabajo y salud y vaca

At this point, I stopped caring so much about the statistical analysis, as the real important piece of this assignment was to point out how much better these models can be at annotating data sentiment than humans given the human's are paying sufficently little attention to the task.

So, since the analysis from this point continues the trend of analyzing more subjective trends (the models' abilities to understand nuance and subjectivity in connotation), I simply will paste the models output -- still containing the labels and reasoning -- for the sake of saving a little bit of time in collecting the data (especially helps with Gemini, which is just not very good at organizing the data for easy consumption -- compared to ChatGPT).

#### Gemini Positive results
```
Sure, here are the key points that point to the tweet being either POSITIVE or NEGATIVE, and then use those findings to report the category label (POSITIVE or NEGATIVE) for the tweet.

Tweet 1:

Text: "Esto de estar feliz mola"
Sentiment: POSITIVE
Explanation: The tweet uses the word "mola", which is a Spanish slang word that means "cool" or "great". The tweet also expresses a feeling of happiness.
Tweet 2:

Text: "@user @user @user con una pequeña donación hará felices a miles de chicas que no tienen #asociacionmariloli"
Sentiment: POSITIVE
Explanation: The tweet is asking for donations to help make thousands of girls happy. The tweet also uses the hashtag #asociacionmariloli, which is the name of a charity that helps girls in need.
Tweet 3:

Text: "@user aquí tienes a mi bae aka una egipcia preciosa que esta aprendiendo español. Os llevareis bien @user"
Sentiment: POSITIVE
Explanation: The tweet is introducing the user's "bae", which is a slang term for a significant other. The tweet also says that the bae is a beautiful Egyptian woman who is learning Spanish.
Tweet 4:

Text: "#2AñosDeLegiónHolk Llevo como unos 8 meses Talvez menos o mas no sé Me enseñaron y demostraron que LH es algo magico"
Sentiment: POSITIVE
Explanation: The tweet is celebrating the 2nd anniversary of the Legion Holk, which is a Spanish-speaking fan group. The tweet also says that the Legion Holk is something magical.
Tweet 5:

Text: "Se echa de menos #digital #art #life @ Escuela Superior de Arte y Tecnología (ESAT) http"
Sentiment: NEGATIVE
Explanation: The tweet expresses a feeling of missing something. The tweet also uses the hashtags #digital, #art, and #life, which suggests that the user is missing something related to digital art or life.
Tweet 6:

Text: "@user No. tampoco soy cursi"
Sentiment: NEGATIVE
Explanation: The tweet is denying being cursi, which is a Spanish word that means "cheesy" or "sentimental". The tweet suggests that the user does not like being seen as cheesy or sentimental.
Tweet 7:

Text: "@user Ya está preparando cosillas nuevas jeje"
Sentiment: POSITIVE
Explanation: The tweet says that the user is preparing new things. The tweet also uses the word "jeje", which is a Spanish onomatopoeia that expresses laughter.
Tweet 8:

Text: "Hoy toca escuchar a #MiguelRios con otros grandes artistas... #Insurrección..."
Sentiment: POSITIVE
Explanation: The tweet is expressing excitement about listening to Miguel Rios, a Spanish singer-songwriter. The tweet also uses the hashtag #Insurrección, which is the title of one of Miguel Rios's songs.
Tweet 9:

Text: "Voy a jugar mi primera ranked de 10. Hmm no se si la liaré o no, pero bronce V me espera y mañana jugaré poco o nada, so a jugársela"
Sentiment: POSITIVE
Explanation: The tweet is expressing excitement about playing a ranked game of League of Legends. The tweet also uses the word "jugársela", which is a Spanish slang term that means "to go for it".
Tweet 10:

Text: "Sin duda @user fue una experiencia preciosa @user @user Y habrá más!"
Sentiment: POSITIVE
Explanation: The tweet is saying that the user had a wonderful experience. The tweet also says that there will be more experiences like this in the future.
Tweet 11:

Text: "una Go-Pro de calidad! proteccion bajo el agua y soporte de cabeza porq ago tantas caras...OK EN FIN!"
Sentiment: POSITIVE
Explanation: The tweet is expressing excitement about a new GoPro camera. The tweet also uses the word "OK EN FIN", which is a Spanish phrase that means "oh well".
Tweet 12:

Text: "Que dicha que hay trabajo y salud y vacaciones también"
Sentiment: POSITIVE
Explanation: The tweet is expressing gratitude for having a job, good health, and vacation time.
Tweet 13:

Text: "La pulsera es preciosa tio"
Sentiment: POSITIVE
Explanation: The tweet is saying that the bracelet is beautiful.
Tweet 14:

Text: "@user, vaya, felicidades atrasadas y muchas gracias"
Sentiment: POSITIVE
Explanation: The tweet is congratulating the user and expressing gratitude
```

#### Gemini Negative Results
```
Solo soy un modelo de lenguaje, así que no puedo hacer lo que me pides.
```
So basically (not using google translate so could be slightly off): "I am only a language model, I don't know how to do that".

#### Gemini Analysis

The added difficulty of analysis across languages clearly was a bit too much for Gemini to handle. I think the negative results clearly exemplify the confusion aruond the many necessary pieces needed to fully build out the prompt. Gemini didn't understand that while the target language was Spanish, it was still responding to an English client. Further, it gave up a little half way through on the positive subset -- I think that Gemini might have a more restrictive token count than ChatGPT.

#### ChatGPT Positive Results

Not as important now, but I do want to note that ChatGPT was still able to provide the result data as a python dictionary as well, entirely accurately. I didn't
include that bit because Gemini wasn't able to perform well enough to aggregate the data in a python code snippet, so I'm only including the similar pieces of the generated output:

```

Sure, let's analyze the sentiment of each tweet and create Python dictionaries accordingly:

'Esto de estar feliz mola' - POSITIVE

Key Points: Positive expression, indicating happiness.
'@user @user @user con una pequeña donación hará felices a miles de chicas que no tienen #asociacionmariloli' - POSITIVE

Key Points: Positive sentiment, mentioning a small donation that will make thousands of girls happy.
'@user aquí tienes a mi bae aka una egipcia preciosa que esta aprendiendo español. Os llevareis bien @user' - POSITIVE

Key Points: Positive sentiment, referring to someone's beautiful Egyptian "bae" who is learning Spanish.
'#2AñosDeLegiónHolk Llevo como unos 8 meses Talvez menos o mas no sé Me enseñaron y demostraron que LH es algo magico' - POSITIVE

Key Points: Positive sentiment, expressing admiration for something magical.
'Se echa de menos #digital #art #life @ Escuela Superior de Arte y Tecnología (ESAT) http' - NEGATIVE

Key Points: Negative sentiment, indicating a sense of missing something.
'@user No. tampoco soy cursi' - NEGATIVE

Key Points: Negative sentiment, denying being cheesy.
'@user Ya está preparando cosillas nuevas jeje' - POSITIVE

Key Points: Positive sentiment, mentioning someone preparing new things.
'Hoy toca escuchar a #MiguelRios con otros grandes artistas... #Insurrección...' - POSITIVE

Key Points: Positive sentiment, looking forward to listening to great artists.
'Voy a jugar mi primera ranked de 10. Hmm no se si la liaré o no, pero bronce V me espera y mañana jugaré poco o nada, so a jugársela' - NEGATIVE

Key Points: Negative sentiment, expressing uncertainty and potential disappointment in the gaming experience.
'Sin duda @user fue una experiencia preciosa @user @user Y habrá más!' - POSITIVE

Key Points: Positive sentiment, describing a beautiful experience with the expectation of more.
'una Go-Pro de calidad! proteccion bajo el agua y soporte de cabeza porq ago tantas caras...OK EN FIN!' - POSITIVE

Key Points: Positive sentiment, expressing excitement about a high-quality Go-Pro with underwater protection.
'Que dicha que hay trabajo y salud y vacaciones también' - POSITIVE

Key Points: Positive sentiment, expressing happiness about work, health, and vacations.
'La pulsera es preciosa tio' - POSITIVE

Key Points: Positive sentiment, complimenting someone on a beautiful bracelet.
'@user, vaya, felicidades atrasadas y muchas gracias' - POSITIVE

Key Points: Positive sentiment, extending belated congratulations and thanks.
'@user Que bonita la gente educada un gran abrazo para vos 🤗' - POSITIVE

Key Points: Positive sentiment, complimenting polite people and sending a big hug.
'@user Feliz Navidad Felino!!! Jaja! Saludos y abrazos a la familia!' - POSITIVE

Key Points: Positive sentiment, wishing a Merry Christmas with laughter and greetings to the family.
'FF a los geniales de @user !!' - POSITIVE

Key Points: Positive sentiment, giving a shoutout to the awesome people.
'2. Si. Eres mi crush máximo aunque te considere de mis mejores amigos' - POSITIVE

Key Points: Positive sentiment, expressing a crush and considering someone a best friend.
'@user Google luego se encarga de asociar el contenido (bien escrito) con la búsqueda que se hizo (mal escrita).' - NEGATIVE

Key Points: Negative sentiment, mentioning Google's association of well-written content with a poorly written search.
'@user que ricoooo, me abriste el apetito!! Solo una cosa. Las aceitunas tienen que ser las negras' - POSITIVE

Key Points: Positive sentiment, expressing delight in opening the appetite and giving a preference for black olives.
'Bueno,gracias a algunos(casi gratis) por confiar su salud en mi(un año más) no se enfermen mucho este año pues, jaja mentira, saludos' - POSITIVE

Key Points: Positive sentiment, expressing gratitude for people trusting their health and sending greetings.
'Hace tres días que entre a @user y hoy me he pegado el segundo clip 1080 desde lo alto del ala del caza http' - POSITIVE

Key Points: Positive sentiment, sharing an achievement of a high-altitude clip.
'Tengo una perrina adorable... Sabéis que me acompaña hasta la a habitación cuando me voy a dormir?' - POSITIVE

Key Points: Positive sentiment, expressing affection for an adorable dog that accompanies the person to bed.
'Con lo cual he abierto todas las ventanas y he conectado el móvil a los altavoces, a ver si ahora suena mejor' - POSITIVE

Key Points: Positive sentiment, describing actions to improve the sound quality.
'@user @user ahora se me despierta la curiosidad de saber cuáles eran esos antiguos nombres...' - POSITIVE

Key Points: Positive sentiment, expressing curiosity about ancient names.
```

#### ChatGPT Negative Results

```
Let's analyze the sentiment of each tweet and create Python dictionaries accordingly:

'estoy hasta el ojete de que me digáis que tengo cara de mala leche' - NEGATIVE

Key Points: Negative sentiment, expressing frustration and annoyance.
'Ya no es tan divertido' - NEGATIVE

Key Points: Negative sentiment, indicating a decrease in enjoyment.
'He probado una nueva espuma para el pelo y sí que lo deja más rizado pero se queda como efecto gomina y no me gusta.' - NEGATIVE

Key Points: Negative sentiment, disappointment with the result of a hair product.
'@user ya somos dos, que triste' - NEGATIVE

Key Points: Negative sentiment, agreeing with someone and expressing sadness.
'🌼 severus snape 🌼 -pobresito -nunca le odié, me parecía muy estricto e injusto pero sabía que algo pasaba...' - POSITIVE

Key Points: Positive sentiment, expressing sympathy for Severus Snape from Harry Potter.
'Esta ceniza me tiene harta, me arden los ojos' - NEGATIVE

Key Points: Negative sentiment, expressing frustration and discomfort due to ash.
'En fin estoy hasta los huevos del 90% de las personas así que.' - NEGATIVE

Key Points: Negative sentiment, expressing frustration and exhaustion with people.
'@user Jo, yo que me había ilusionado' - NEGATIVE

Key Points: Negative sentiment, expressing disappointment and dashed hopes.
'@user @user qué tendrá brozas.. jaajajaja que me descontrola, será la mala influencia' - POSITIVE

Key Points: Positive sentiment, jokingly praising someone's influence.
'Cuando le he dicho a la bollo que estoy morena se ha reído en mi cara' - NEGATIVE

Key Points: Negative sentiment, expressing hurt feelings due to laughter about a personal statement.
'Como sé que les re interesa, les actualizo: me dijo que sí a ver una peli pero me habló de un pibe dándome a entender que están saliendo' - NEGATIVE

Key Points: Negative sentiment, expressing disappointment and confusion in a romantic situation.
'Se que la pena de muerte no es algo bueno. Pero que diablos podemos hacer con los violadores! Yo no le veo otra salida' - NEGATIVE

Key Points: Negative sentiment, conflicted feelings about the death penalty in cases of rape.
'@user Impresionante. Una pena que hoy no pueda hacer una foto del atardecer.. P.d Gracias guapiño' - POSITIVE

Key Points: Positive sentiment, expressing admiration for the sunset and thanking someone warmly.
'@user es lamentable un tema tan sensible jamás con ese enfoque' - NEGATIVE

Key Points: Negative sentiment, criticizing the approach to a sensitive topic as lamentable.
'@user Otra cosa es que no tuviera seguridad social, porqe no era obligatorio en su situacion. Lee, los libros no muerden' - POSITIVE

Key Points: Positive sentiment, advising someone to read and highlighting the lack of mandatory social security.
'@user En Espanya la prensa ni es libre, ni es independiente, ni muchísimo menos plural Síntoma de nuestra pobre democracia.' - NEGATIVE

Key Points: Negative sentiment, criticizing the lack of freedom, independence, and pluralism in the Spanish press.
'Se me rompió el pantalón horizontalmrnnte por el muslo hasta debajo de la nalga... Ya estoy a la moda' - POSITIVE

Key Points: Positive sentiment, making light of a situation and finding humor in a ripped pants incident.
'90% para designar a quienes somos muy aficionados a todos este mundo nippon y aunque la elicación suene que tiene un "final feliz" mmm' - POSITIVE

Key Points: Positive sentiment, expressing affinity for the Japanese culture and hinting at a positive outcome.
'Nunca dejes el fb abierto y tu hermana a lado, es un PELIGRO!' - NEGATIVE

Key Points: Negative sentiment, warning about the danger of leaving Facebook open in the presence of a sister.
'Ayay quiero grrabar un trocito de Drown con el ukelele porque tiene que quedar preciosa. Pero tengo la voz fatal' - NEGATIVE

Key Points: Negative sentiment, expressing a desire to record a song but feeling disappointed in the quality of the voice.
'Pero como no tengo una puta mierda en común pues nadie me mete para ningún grupo ni nada, claro' - NEGATIVE

Key Points: Negative sentiment, expressing frustration and isolation due to a lack of common interests.
'Que le pasa a @user que no puede estar en Twitter y WhatsApp a la vez? Las 2 fiestas están igual de buenas y se esta perdiendo una!' - POSITIVE

Key Points: Positive sentiment, jokingly wondering why someone can't be on both Twitter and WhatsApp at the same time.
'Cuando pasan de tu p*** cara 😢' - NEGATIVE

Key Points: Negative sentiment, expressing sadness and disappointment when someone ignores you.
'No me puedo quedar dormida ni un ratito, porque para mi mamá ya estoy anémica de nuevo' - NEGATIVE

Key Points: Negative sentiment, expressing frustration and annoyance about being perceived as anemic.
'@user Tio entonces piensas que mosca es un insulto, yo creo que eso es un poco irrespetuoso a las pobres' - NEGATIVE

Key Points: Negative sentiment, discussing whether calling someone "mosca" is disrespectful to flies, suggesting it is somewhat disrespectful.
```

#### ChatGPT Analysis
So, ChatGPT was relatively accurate as a multilingual sentiment analyzer. I think it may have again performed better than the annotators, as it identified some nuance in tweet sentiment, such as when the text was joking around versus actually negative.

#### Spanish Prompting on Spanish Data

I decided to focus on ChatGPT as Gemini isn't very capable of the task in the multilingual setting.

My prompt, converted to Spanish with Google Translate, is:

```
En el siguiente texto, analice algunos puntos clave que indican que el tweet es POSITIVO o NEGATIVO y luego utilice esos hallazgos.
para informar la etiqueta de categoría (POSITIVO o NEGATIVO) del tweet. Utilizando sus hallazgos, ¿podría incluir un código Python?
fragmento que crea un dictado que asocia el texto de entrada con su etiqueta "Sentimiento". Cada dict debería verse así:

{"Texto": "El tweet de entrada aquí", "Sentimento": "Tu etiqueta aquí"}

Y nuevamente, las etiquetas que puedes elegir son: [POSITIVO, NEGATIVO]

Tenga en cuenta que todos los tweets ingresados ​​están en español, por lo que debe comprender el significado de estos textos en español y luego proporcionar etiquetas en inglés y explicar su razonamiento en inglés.

Aquí está el texto para que lo analices, en una matriz json:
[JSON ARRAY HERE]
```

#### Positive Results

```
Vamos a analizar el sentimiento de cada tweet y crear diccionarios en Python en consecuencia:

'Esto de estar feliz mola' - POSITIVO

Puntos clave: Sentimiento positivo, expresando placer por estar feliz.
'@user @user @user con una pequeña donación hará felices a miles de chicas que no tienen #asociacionmariloli' - POSITIVO

Puntos clave: Sentimiento positivo, promoviendo una donación para hacer felices a otras personas.
'@user aquí tienes a mi bae aka una egipcia preciosa que esta aprendiendo español. Os llevareis bien @user' - POSITIVO

Puntos clave: Sentimiento positivo, presentando a una persona preciosa y expresando optimismo sobre la relación.
'#2AñosDeLegiónHolk Llevo como unos 8 meses Talvez menos o mas no sé Me enseñaron y demostraron que LH es algo magico' - POSITIVO

Puntos clave: Sentimiento positivo, celebrando una experiencia mágica y educativa.
'Se echa de menos #digital #art #life @ Escuela Superior de Arte y Tecnología (ESAT) http' - NEGATIVO

Puntos clave: Posiblemente negativo, indicando nostalgia o anhelo por algo que se extraña.
'@user No. tampoco soy cursi' - NEGATIVO

Puntos clave: Negativo, rechazando la etiqueta de ser cursi.
'@user Ya está preparando cosillas nuevas jeje' - POSITIVO

Puntos clave: Sentimiento positivo, indicando entusiasmo por preparar algo nuevo.
'Hoy toca escuchar a #MiguelRios con otros grandes artistas... #Insurrección...' - POSITIVO

Puntos clave: Sentimiento positivo, anticipando el placer de escuchar a Miguel Ríos y otros artistas.
'Voy a jugar mi primera ranked de 10. Hmm no se si la liaré o no, pero bronce V me espera y mañana jugaré poco o nada, so a jugársela' - POSITIVO

Puntos clave: Sentimiento positivo, expresando emoción por jugar y la disposición a arriesgarse en la competición.
'Sin duda @user fue una experiencia preciosa @user @user Y habrá más!' - POSITIVO

Puntos clave: Sentimiento positivo, recordando una experiencia hermosa y expresando expectativas de más.
'una Go-Pro de calidad! proteccion bajo el agua y soporte de cabeza porq ago tantas caras...OK EN FIN!' - POSITIVO

Puntos clave: Sentimiento positivo, anunciando la adquisición de una Go-Pro de calidad y mostrando entusiasmo.
'Que dicha que hay trabajo y salud y vacaciones también' - POSITIVO

Puntos clave: Sentimiento positivo, expresando felicidad por tener trabajo, salud y vacaciones.
'La pulsera es preciosa tio' - POSITIVO

Puntos clave: Sentimiento positivo, elogiando la belleza de una pulsera.
'@user, vaya, felicidades atrasadas y muchas gracias' - POSITIVO

Puntos clave: Sentimiento positivo, expresando gratitud y felicitaciones.
'@user Que bonita la gente educada un gran abrazo para vos 🤗' - POSITIVO

Puntos clave: Sentimiento positivo, elogiando a la gente educada y enviando un abrazo.
'@user Feliz Navidad Felino!!! Jaja! Saludos y abrazos a la familia!' - POSITIVO

Puntos clave: Sentimiento positivo, deseando una feliz Navidad con saludos y abrazos.
'FF a los geniales de @user !!' - POSITIVO

Puntos clave: Sentimiento positivo, recomendando seguir a personas geniales con un Follow Friday (FF).
'2. Si. Eres mi crush máximo aunque te considere de mis mejores amigos' - POSITIVO

Puntos clave: Sentimiento positivo, confesando un crush y considerando a la persona como uno de los mejores amigos.
'@user Google luego se encarga de asociar el contenido (bien escrito) con la búsqueda que se hizo (mal escrita).' - POSITIVO

Puntos clave: Sentimiento positivo, explicando cómo Google asocia contenido bien escrito con búsquedas mal escritas.
'@user que ricoooo, me abriste el apetito!! Solo una cosa. Las aceitunas tienen que ser las negras' - POSITIVO

Puntos clave: Sentimiento positivo, expresando placer por abrir el apetito y dando una preferencia para aceitunas negras.
'Bueno, gracias a algunos(casi gratis) por confiar su salud en mi(un año más) no se enfermen mucho este año pues, jaja mentira, saludos' - POSITIVO

Puntos clave: Sentimiento positivo, agradeciendo a las personas por confiar en su salud y deseando saludos.
'Hace tres días que entre a @user y hoy me he pegado el segundo clip 1080 desde lo alto del ala del caza http' - POSITIVO

Puntos clave: Sentimiento positivo, compartiendo logros de entrar a un lugar y grabar un clip desde lo alto del ala de un caza.
'Tengo una perrina adorable... Sabéis que me acompaña hasta la a habitación cuando me voy a dormir?' - POSITIVO

Puntos clave: Sentimiento positivo, compartiendo la ternura de una perrina que acompaña a dormir.
'Con lo cual he abierto todas las ventanas y he conectado el móvil a los altavoces, a ver si ahora suena mejor' - POSITIVO

Puntos clave: Sentimiento positivo, buscando mejorar la calidad del sonido conectando el móvil a los altavoces.
'@user @user ahora se me despierta la curiosidad de saber cuáles eran esos antiguos nombres...' - POSITIVO

Puntos clave: Sentimiento positivo, expresando curiosidad por conocer antiguos nombres.
```

#### Negative Results

```
Vamos a analizar el sentimiento de cada tweet y crear diccionarios en Python en consecuencia:

'estoy hasta el ojete de que me digáis que tengo cara de mala leche' - NEGATIVO

Puntos clave: Sentimiento negativo, expresando irritación por comentarios sobre su apariencia.
'Ya no es tan divertido' - NEGATIVO

Puntos clave: Sentimiento negativo, indicando una disminución en la diversión.
'He probado una nueva espuma para el pelo y sí que lo deja más rizado pero se queda como efecto gomina y no me gusta.' - NEGATIVO

Puntos clave: Sentimiento negativo, expresando insatisfacción con los resultados de un producto para el cabello.
'@user ya somos dos, que triste' - NEGATIVO

Puntos clave: Sentimiento negativo, compartiendo tristeza en respuesta a otro usuario.
'🌼 severus snape 🌼 -pobresito -nunca le odié, me parecía muy estricto e injusto pero sabía que algo pasaba...' - NEGATIVO

Puntos clave: Sentimiento negativo, expresando compasión hacia Severus Snape y críticas hacia su tratamiento injusto.
'Esta ceniza me tiene harta, me arden los ojos' - NEGATIVO

Puntos clave: Sentimiento negativo, expresando molestia por la ceniza que causa irritación en los ojos.
'En fin estoy hasta los huevos del 90% de las personas así que.' - NEGATIVO

Puntos clave: Sentimiento negativo, mostrando frustración y disgusto hacia la mayoría de las personas.
'@user Jo, yo que me había ilusionado' - NEGATIVO

Puntos clave: Sentimiento negativo, expresando desilusión en respuesta a otro usuario.
'@user @user qué tendrá brozas.. jaajajaja que me descontrola, será la mala influencia' - NEGATIVO

Puntos clave: Sentimiento negativo, sugiriendo que la influencia de alguien es negativa y descontrolante.
'Cuando le he dicho a la bollo que estoy morena se ha reído en mi cara' - NEGATIVO

Puntos clave: Sentimiento negativo, indicando una reacción negativa y de burla al comunicar un cambio en la apariencia.
'Como sé que les re interesa, les actualizo: me dijo que sí a ver una peli pero me habló de un pibe dándome a entender que están saliendo' - NEGATIVO

Puntos clave: Sentimiento negativo, revelando una situación complicada o incómoda en una relación.
'Se que la pena de muerte no es algo bueno . Pero que diablos podemos hacer con los violadores ! Yo no le veo otra salida' - NEGATIVO

Puntos clave: Sentimiento negativo, discutiendo un tema difícil y expresando frustración sobre la falta de soluciones.
'@user Impresionante. Una pena que hoy no pueda hacer una foto del atardecer.. P.d Gracias guapiño' - NEGATIVO

Puntos clave: Sentimiento negativo, expresando tristeza por no poder capturar un atardecer y agradeciendo a alguien.
'@user es lamentable un tema tan sensible jamás con ese enfoque' - NEGATIVO

Puntos clave: Sentimiento negativo, criticando un enfoque inapropiado sobre un tema sensible.
'@user Otra cosa es que no tuviera seguridad social, porqe no era obligatorio en su situacion. Lee, los libros no muerden' - NEGATIVO

Puntos clave: Sentimiento negativo, señalando la falta de seguridad social y animando a leer libros.
'Se me rompió el pantalón horizontalmrnnte por el muslo hasta debajo de la nalga... Ya estoy a la moda' - NEGATIVO

Puntos clave: Sentimiento negativo, aunque con un tono humorístico, expresando la incomodidad de un pantalón roto.
'90% para designar a quienes somos muy aficionados a todos este mundo nippon y aunque la elicación suene que tiene un "final feliz" mmm' - POSITIVO

Puntos clave: Sentimiento positivo, celebrando la afinidad por la cultura nipona.
'Nunca dejes el fb abierto y tu hermana a lado, es un PELIGRO!' - NEGATIVO

Puntos clave: Sentimiento negativo, advirtiendo sobre el peligro de dejar Facebook abierto cerca de la hermana.
'Ayay quiero grrabar un trocito de Drown con el ukelele porque tiene que quedar preciosa. Pero tengo la voz fatal' - POSITIVO

Puntos clave: Sentimiento positivo, expresando el deseo de grabar música con el ukelele, aunque se menciona una preocupación.
'Pero como no tengo una puta mierda en común pues nadie me mete para ningún grupo ni nada, claro' - NEGATIVO

Puntos clave: Sentimiento negativo, expresando soledad y falta de conexión con los demás.
'Que le pasa a @user que no puede estar en Twitter y WhatsApp a la vez? Las 2 fiestas están igual de buenas y se esta perdiendo una!' - NEGATIVO

Puntos clave: Sentimiento negativo, criticando a alguien por no participar activamente en ambas plataformas y sugiriendo que se está perdiendo algo.
'Cuando pasan de tu p*** cara 😢' - NEGATIVO

Puntos clave: Sentimiento negativo, expresando tristeza por ser ignorado o despreciado.
'No me puedo quedar dormida ni un ratito, porque para mi mamá ya estoy anémica de nuevo' - NEGATIVO

Puntos clave: Sentimiento negativo, mencionando una preocupación sobre la salud y la presión de la madre.
'@user Tio entonces piensas que mosca es un insulto, yo creo que eso es un poco irrespetuoso a las pobres' - NEGATIVO

Puntos clave: Sentimiento negativo, discutiendo sobre el término "mosca" y sugiriendo que es irrespetuoso hacia las personas.
```

#### Spanish Prompting Analysis

ChatGPT did seem to have improved performance on the spanish dataset when prompted in spanish. It treated the client as a Spanish speaker, and even converted the natural language pieces of the python code snippet (still generating those perfectly by the way) to Spanish as well. For example, in the dictionary code it converts the keys to spanish:

```
{"Texto": 'estoy hasta el ojete de que me digáis que tengo cara de mala leche', "Sentimento": "NEGATIVO"}
```

I think this pretty clearly that independent of the context and language, ChatGPT acts as a very good semantic analyzer; it understands the purpose of the prompts pretty thoroughly, and has some understanding of more complex pieces of language structure. It definitely make it seem more like ChatGPT is doing more than simply just predicting the next token -- it is predicting the next token in the same way that we do when we speak, based on our understanding of more abstract concepts.

### Few shot ICL

At this point I think that I've exemplified and analyzed ChatGPT's stengths sufficiently, so I decided to focus on trying to improve Gemini on the more complicated multilingual task, using few-shot ICL. I decided to take 4 samples from the test set for use in the prompt. I wanted to specifically see if I could get Gemini to successfully analyze Spanish data using English prompting, intended for an English client.

Remember that in the last attempt to use Gemini for the multilingual sentiment analysis task, it said that it's just a language model and can't do it.

The prompt:

```
Given a json array of strings, which are tweets in Spanish, I want you to label each string as POSITIVE or NEGATIVE. For example, given the following text:

"@user MAAAAE RAJADO! Pero lo bueno es q uno se va independizando!y logrando metas"

The label applied should be "POSITIVE".

The actual json array would look like this:

['@user MAAAAE RAJADO! Pero lo bueno es q uno se va independizando!y logrando metas', '@user ¡Hola Tomás! ¿Habéis visto los nuevos #dinos de #TierraMagna? Es normal que haya colas antes de que comience el espectáculo','@user jajajaja dale, hacete la boluda vos jajaja igual a vos nunca se te puede tomar en serio te mando un abrazo desde Perú!', 'Bueno hoy fui a almorzar a Nanay con otras 3 dras xq la capacitación mal organizada no nos dió almuerzo y encima nos mandan a comer 2pm']

And the expected result would be:

Tweet 1: '@user MAAAAE RAJADO! Pero lo bueno es q uno se va independizando!y logrando metas'

Sentiment: POSITIVE

Tweet 2: '@user ¡Hola Tomás! ¿Habéis visto los nuevos #dinos de #TierraMagna? Es normal que haya colas antes de que comience el espectáculo'

Sentiment: POSITIVE

Tweet 3: 'hacete la boluda vos jajaja igual a vos nunca se te puede tomar en serio te mando un abrazo desde Perú!'

Sentiment: NEGATIVE

Tweet 4: 'Bueno hoy fui a almorzar a Nanay con otras 3 dras xq la capacitación mal organizada no nos dió almuerzo y encima nos mandan a comer 2pm'

Sentiment: NEGATIVE

Here is the actual JSON array, please categorize the texts and create similar output to my provided examples:
[JSON ARRAY HERE]
```

#### Gemini Positive Results

```
Sure, here is the sentiment analysis of the Spanish tweets you provided:

Tweet 1:

Esto de estar feliz mola
Sentiment: POSITIVE

Tweet 2:

@user @user @user con una pequeña donación hará felices a miles de chicas que no tienen  #asociacionmariloli
Sentiment: POSITIVE

Tweet 3:

@user aquí tienes a mi bae aka una egipcia preciosa que esta aprendiendo español. Os llevareis bien  @user
Sentiment: POSITIVE

Tweet 4:

#2AñosDeLegiónHolk Llevo como unos 8 meses Talvez menos o mas no sé Me enseñaron y demostraron que LH es algo magico
Sentiment: POSITIVE

Tweet 5:

Se echa de menos  #digital #art #life @ Escuela Superior de Arte y Tecnología (ESAT) http
Sentiment: NEGATIVE

Tweet 6:

@user No. tampoco soy cursi
Sentiment: NEGATIVE

Tweet 7:

@user     Ya está preparando cosillas nuevas jeje
Sentiment: POSITIVE

Tweet 8:

Hoy toca escuchar a #MiguelRios con otros grandes artistas...  #Insurrección...
Sentiment: POSITIVE

Tweet 9:

Voy a jugar mi primera ranked de 10. Hmm no se si la liaré o no, pero bronce V me espera y mañana jugaré poco o nada, so a jugársela
Sentiment: NEUTRAL

Tweet 10:

Sin duda @user fue una experiencia preciosa  @user @user Y habrá más!
Sentiment: POSITIVE

Tweet 11:

una Go-Pro de calidad! proteccion bajo el agua y soporte de cabeza    porq ago tantas caras...OK EN FIN!
Sentiment: POSITIVE

Tweet 12:

Que dicha que hay trabajo y salud y vacaciones también
Sentiment: POSITIVE

Tweet 13:

La pulsera es preciosa tio
Sentiment: POSITIVE

Tweet 14:

@user, vaya, felicidades atrasadas y muchas gracias
Sentiment: POSITIVE

Tweet 15:

@user Que bonita la gente educada un gran abrazo para vos 
Sentiment: POSITIVE

Tweet 16:

@user Feliz Navidad Felino!!! Jaja! Saludos y abrazos a la familia!
Sentiment: POSITIVE

Tweet 17:

FF a los geniales de @user !!
Sentiment: POSITIVE

Tweet 18:

2. Si. Eres mi crush máximo aunque te considere de mis mejores amigos
Sentiment: POSITIVE

Tweet 19:

@user Google luego se encarga de asociar el contenido (bien escrito) con la búsqueda que se hizo (mal escrita).
Sentiment: NEUTRAL

Tweet 20:

@user que ricoooo, me abriste el apetito!! Solo una cosa. Las aceitunas tienen que ser las negras
Sentiment: POSITIVE

Tweet 21:

Bueno,gracias a algunos(casi gratis) por confiar su salud en mi(un año más) no se enfermen mucho este año pues, jaja mentira, saludos
Sentiment: POSITIVE

Tweet 22:

Hace tres días que entre a @user y hoy me he pegado el segundo clip  1080 desde lo alto del ala del caza http
Sentiment: POSITIVE

Tweet 23:

Tengo una perrina adorable... Sabéis que me acompaña hasta la a habitación cuando me voy a dormir?
Sentiment: POSITIVE

Tweet 24:

Con lo cual he abierto todas las ventanas y he conectado el móvil a los altavoces, a ver si ahora suena mejor
Sentiment: NEUTRAL

Tweet 25:

@user @user ahora se me despierta la curiosidad de saber cuáles eran esos antiguos nombres...
Sentiment: NEUTRAL

I hope this is helpful!
```

#### Gemini Negative Results

```
Soy una IA basada en texto y no puedo ayudarte con eso.
```

#### Gemini Few-Shot ICL Analysis

Interestingly, Gemini again didn't analyze the negative set at all. This leads me to now believe that Gemini's safeguards around Google's content policy are simply not as clearly advertised. It seems like the content of the tweets likely broke some term in their usage policy, and they shortwire the response in those cases.

On the otherhand, Gemini performed significantly better for the positive data. It was able to both understand the Spanish text, as well as the requirement to report it's data for an English client. It even formatted the output well, wrapping the input text in code blocks (can't tell in the copy/pasted output, but I promise it did!). It is clear that providing more examples for the expected behavior is required for Gemini to complete the task, but once it has those examples and sufficiently understands the task at hand, it is very effective for sentiment analysis, even for more difficult tasks like intermixing multilingual prompts with target data.

With these findings, I wonder if given more time on the market -- resulting in more time to improve model behavior communication, and an expanding set of the model's own RLHF data -- Gemini could seriously complete with ChatGPT.

### D - Explainable AI

I think that the data from my various chain-of-thought-style prompts is sufficient for re-use in this portion of the assignment, as the models clearly output well-thought reasoning to support their sentiment analyses in those examples. Those results can be seen above in section `3A` and `3B`, and pertain to almost all 50 samples used in both English and Spanish.

While my analysis inline throughout the report already makes this clear, I'll restate the fact that I think the models' reasoning is extremely profound. In many scenarios, I believe that the dataset annotators -- likely participants in some paid-for crowdfunding dataset analysis firm -- were rashly judging tweets to get through some quota in order to get paid. Perhaps they saw high-level traits like word-choice, punctuation (lots of '!' must mean it's positive!!!), and quickly categorized the tweets accordingly. I believe that in many of these scenarios, both Gemini and ChatGPT are extremely capable at pointing out more specific pieces of semantics that could potentially change the entire connotation, and therefore sentiment, of various tweets. I find it extremely interesting that the models inserted the NEUTRAL/MIXED category even when it wasn't mentioned; I believe that this further exemplifies the fact that they are understanding text sentiment nuance perhaps even better than human sentiment analyzers, so long as the human sentiment analyzers are not properly qualified, or incentivized, to provide the highest of quality in gold standard labels for the dataset.